In [1]:
import collections
import numpy as np

In [2]:
def get_words(message):
    return [word.lower() for word in message.split()]


def create_dictionary(messages):
    temp = {}
    mapping = {}
    i = 0
    for m in messages:
        words = get_words(m)
        for w in words:
            if w not in temp:
                temp[w] = 0
            else:
                temp[w] += 1
            if temp[w] == 5:
                mapping[w] = i
                i += 1
    return mapping


def transform_text(messages, word_dictionary):
    N, V = len(messages), len(word_dictionary)
    data = np.zeros((N, V))
    for i, m in enumerate(messages):
        for w in get_words(m):
            if w in word_dictionary:
                data[i, word_dictionary[w]] += 1
    return data

In [3]:
# Naive Bayes

def fit_naive_bayes_model(matrix, labels):
    # matrix (M, V)
    labels = labels.astype(np.bool)
    M, V = matrix.shape
   
    # frequency list, (2, |V|)
    phi_k = np.zeros((2, V))
    
    # retrieve all ham messages
    phi_k[0] = matrix[~labels].sum(axis=0)
    phi_k[1] = matrix[labels].sum(axis=0)
    
    # add one laplace smoothing
    phi_k += 1
    
    # normalize
    phi_k = phi_k / phi_k.sum(axis=1, keepdims=True)
    phi_y = labels.sum() / labels.shape[0]
    
    return phi_k, phi_y
    

def predict_from_naive_bayes_model(model, matrix):
    # matrix (M, V)
    # (2, V), scalar
    phi_k, phi_y = model

    # log likelihood (M, 2)
    log_likelihood = np.sum(matrix[:,None] * np.log(phi_k), axis=-1)
    log_likelihood[:, 0] += np.log(1 - phi_y)
    log_likelihood[:, 1] += np.log(phi_y)
    
    return np.argmax(log_likelihood, axis=1)


def get_top_five_naive_bayes_words(model, dictionary):
    # (2, V)
    phi_k, phi_y = model
    ratio = phi_k[1] / phi_k[0]
    indices = np.argsort(ratio)[-5:][::-1]  
    idx2word = {v: k for k, v in dictionary.items()}
    words = [idx2word[idx] for idx in indices]
    return words

In [4]:
# SVM

np.random.seed(42)
def train_and_predict_svm(train_matrix, train_labels, test_matrix, radius):
    model = svm_train(train_matrix, train_labels, radius)
    return svm_predict(model, test_matrix, radius)

def svm_train(matrix, category, radius):
    state = {}
    M, N = matrix.shape

    Y = 2*category-1
    matrix = 1.0*(matrix > 0)
    squared = np.sum(matrix*matrix, axis=1)
    gram = matrix.dot(matrix.T)
    K = np.exp(-(squared.reshape((1, -1)) + squared.reshape((-1, 1)) - 2 * gram) / (2 * (radius ** 2)) )

    alpha = np.zeros(M)
    alpha_avg = np.zeros(M)
    L = 1.0/(64*M)
    outer_loops = 10

    alpha_avg
    for ii in range(outer_loops * M):
        i = int(np.random.rand() * M)
        margin = Y[i] * np.dot(K[i, :], alpha)
        grad = M * L * K[:, i] * alpha[i]
        if (margin < 1):
            grad -=  Y[i] * K[:, i]
        alpha -=  grad / np.sqrt(ii + 1)
        alpha_avg += alpha

    alpha_avg /= (ii + 1) * M

    state["alpha"] = alpha
    state["alpha_avg"] = alpha_avg
    state["Xtrain"] = matrix
    state["Sqtrain"] = squared

    return state


def svm_predict(state, matrix, radius):
    M, N = matrix.shape
    output = np.zeros(M)

    Xtrain = state["Xtrain"]
    Sqtrain = state["Sqtrain"]
    matrix = 1. * (matrix > 0)
    squared = np.sum(matrix * matrix, axis=1)
    gram = matrix.dot(Xtrain.T)
    K = np.exp(-(squared.reshape((-1, 1)) + Sqtrain.reshape((1, -1)) - 2 * gram) / (2 * (radius ** 2)))
    alpha_avg = state["alpha_avg"]
    preds = K.dot(alpha_avg)
    output = (1 + np.sign(preds)) // 2

    return output


def compute_best_svm_radius(train_matrix, train_labels, val_matrix, val_labels, radius_to_consider):
    best_radius, best_acc = 0.0, 0.0

    for r in radius_to_consider:
        pred = train_and_predict_svm(train_matrix, train_labels, val_matrix, r)
        acc = (pred == val_labels).sum() / len(pred)
        if acc > best_acc:
            best_radius = r
            best_acc = acc
    return best_radius

In [5]:
import pandas as pd

#train dataset
train = pd.read_csv("/content/drive/My Drive/Colab Notebooks/CS229/ps2/data/ds6_train.tsv", delimiter="\t", header=None, names=["labels", "messages"])
train_messages = []
train_labels = []
for l, m in zip(train["labels"], train["messages"]):
    train_messages.append(m)
    train_labels.append(1 if l == 'spam' else 0)
train_labels = np.array(train_labels)

#val dataset
val = pd.read_csv("/content/drive/My Drive/Colab Notebooks/CS229/ps2/data/ds6_val.tsv", sep="\t", header=None, names=["labels", "messages"])
val_messages = []
val_labels = []
for l, m in zip(val["labels"], val["messages"]):
    val_messages.append(m)
    val_labels.append(1 if l == 'spam' else 0)
val_labels = np.array(val_labels)

#test dataset
test = pd.read_csv("/content/drive/My Drive/Colab Notebooks/CS229/ps2/data/ds6_test.tsv", sep="\t", header=None, names=["labels", "messages"])
test_messages = []
test_labels = []
for l, m in zip(test["labels"], test["messages"]):
    test_messages.append(m)
    test_labels.append(1 if l == 'spam' else 0)
test_labels = np.array(test_labels)

#dictionary
dictionary = create_dictionary(train_messages)

#write json
#import json
#with open("/content/drive/My Drive/Colab Notebooks/CS229/ps2/output/p06_dictionary", "w") as f:
#    json.dump(dictionary, f)

#matrix
train_matrix = transform_text(train_messages, dictionary)
#np.savetxt("/content/drive/My Drive/Colab Notebooks/CS229/ps2/output/p06_sample_train_matrix", train_matrix[:100,:])

#val
val_matrix = transform_text(val_messages, dictionary)
test_matrix = transform_text(test_messages, dictionary)

#naive bayes
naive_bayes_model = fit_naive_bayes_model(train_matrix, train_labels)
naive_bayes_predictions = predict_from_naive_bayes_model(naive_bayes_model, test_matrix)
#np.savetxt("/content/drive/My Drive/Colab Notebooks/CS229/ps2/output/p06_naive_bayes_predictions", naive_bayes_predictions)
naive_bayes_accuracy = np.mean(naive_bayes_predictions == test_labels)
print("Naive Bayes had an accuracy of {} on the testing set".format(naive_bayes_accuracy))

#top 5
top_5_words = get_top_five_naive_bayes_words(naive_bayes_model, dictionary)
print("\nThe top 5 indicative words for Naive Bayes are: ", top_5_words)

#svm
optimal_radius = compute_best_svm_radius(train_matrix, train_labels, val_matrix, val_labels, [0.01, 0.1, 1, 10])
print("\nThe optimal SVM radius was {}".format(optimal_radius))

svm_predictions = train_and_predict_svm(train_matrix, train_labels, test_matrix, optimal_radius)
svm_accuracy = np.mean(svm_predictions == test_labels)

print("\nThe SVM model had an accuracy of {} on the testing set".format(svm_accuracy, optimal_radius))

Naive Bayes had an accuracy of 0.978494623655914 on the testing set

The top 5 indicative words for Naive Bayes are:  ['claim', 'won', 'prize', 'tone', 'urgent!']

The optimal SVM radius was 0.1

The SVM model had an accuracy of 0.967741935483871 on the testing set
